In [1]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertModel
from transformers import get_linear_schedule_with_warmup

SEED = 123
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 1e-2
EPSILON = 1e-8
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


In [2]:
def read_imdb(data_dir):
    """读取IMDb评论数据集文本序列和标签"""
    data, labels = [], []
    df = pd.read_csv(data_dir).loc[:5000]
    data_list = df.loc[:,'discourse_text'].values
    label_list_ = df.loc[:,'discourse_type'].values
    type_dict = {'Lead':0,'Position':1,'Claim':2,'Counterclaim':3,'Rebuttal':4,'Evidence':5,'Concluding Statement':6}
    label_list = []
    for i in label_list_:
        label_list.append(type_dict[i])
    if len(data_list)!=len(label_list):
        return 'length bug'
    else:
        n = len(data_list)
        for data_ in range(n):
            sentence_split = data_list[data_].split(".")[:-1] if data_list[data_].split('.')[-1] == str('') else data_list[data_].split(".")
            label_split = [label_list[data_] for i in range(len(sentence_split))]
            for j in range(len(sentence_split)):
                if sentence_split[j]!=' ':
                    data.append(sentence_split[j]) 
                    labels.append(label_split[j])
        return data, labels

train_data = read_imdb('./train.csv')
print('训练集数目：', len(train_data[0]))
for x, y in zip(train_data[0][:50], train_data[1][:50]):
    print('标签：', y, 'review:', x[0:100])

训练集数目： 12971
标签： 0 review: Modern humans today are always on their phone
标签： 0 review:  They are always on their phone more than 5 hours a day no stop 
标签： 0 review: All they do is text back and forward and just have group Chats on social media
标签： 0 review:  They even do it while driving
标签： 1 review: They are some really bad consequences when stuff happens when it comes to a phone
标签： 5 review: Some certain areas in the United States ban phones from class rooms just because of it
标签： 5 review: When people have phones, they know about certain apps that they have 
标签： 5 review: Apps like Facebook Twitter Instagram and Snapchat
标签： 5 review:  So like if a friend moves away and you want to be in contact you can still be in contact by posting
标签： 5 review:  People always have different ways how to communicate with a phone
标签： 5 review:  Phones have changed due to our generation
标签： 2 review: Driving is one of the way how to get around
标签： 2 review:  People always be on their phones while 

In [3]:
sentences = train_data[0]
labels = train_data[1]
def lower(input_list):
    output_list = []
    for i in input_list:
        output_list.append(i.lower())
    return output_list
sentences = lower(sentences)

In [4]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [5]:
'''
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)
'''

"\nmax_len = 0\n\n# For every sentence...\nfor sent in sentences:\n\n    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.\n    input_ids = tokenizer.encode(sent, add_special_tokens=True)\n    \n    # Update the maximum sentence length.\n    max_len = max(max_len, len(input_ids))\n\nprint('Max sentence length: ', max_len)\n"

In [6]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  modern humans today are always on their phone
Token IDs: tensor([ 101, 2715, 4286, 2651, 2024, 2467, 2006, 2037, 3042,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])


In [7]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

11,673 training samples
1,298 validation samples


In [8]:
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=16)

test_sampler = SequentialSampler(val_dataset)
test_dataloader = DataLoader(val_dataset, sampler=test_sampler, batch_size=16)

In [9]:
for i, (train, mask, label) in enumerate(train_dataloader):
    print(train.shape, mask.shape, label.shape)               #torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16, 1])
    break
print('len(train_dataloader)=', len(train_dataloader))        #500

torch.Size([16, 32]) torch.Size([16, 32]) torch.Size([16])
len(train_dataloader)= 730


In [47]:
import torch.nn as nn
class BertClassification(nn.Module):
    def __init__(self,model_name,num_labels,freeze_bert=False):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.num_labels = num_labels
        D_in, D_out = self.bert.config.hidden_size, num_labels
        self.outlayer = nn.Sequential(nn.Tanh(),nn.Dropout(0.1),nn.Linear(D_in,D_out))
        self.loss = nn.CrossEntropyLoss()
        if freeze_bert==True:
            print('freeze bert para')
            for paras in self.bert.parameters():
                paras.requrie_grad = False
    def forward(self,input_ids,attention_masks,labels=None):
        out = self.bert(input_ids,attention_masks)
        last_hidden_state_cls = out[0][:, 0, :]
        logits = self.outlayer(last_hidden_state_cls)
        if labels is not None:
            predictions = torch.sigmoid(logits)
            loss = self.loss(
                #predictions.view(-1, self.num_labels), labels.view(-1, self.num_labels))
                predictions,labels)
            return loss,logits
        else:
            return 0, logits
        

In [48]:
#model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels = 7)     #num_labels表示2个分类，好评和差评
model = BertClassification('bert-base-uncased',7,freeze_bert=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [49]:
optimizer = AdamW(model.parameters(), lr = 3e-5, eps = 1e-8)

In [50]:
epochs = 3
# training steps 的数量: [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs
# 设计 learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)


In [51]:
def binary_acc(preds, labels):      #preds.shape=(16, 2) labels.shape=torch.Size([16, 1])
    correct = torch.eq(torch.max(preds, dim=1)[1], labels.flatten()).float()      #eq里面的两个参数的shape=torch.Size([16])    
    acc = correct.sum().item() / len(correct)
    return acc

In [52]:
import time
import datetime
def format_time(elapsed):    
    elapsed_rounded = int(round((elapsed)))    
    return str(datetime.timedelta(seconds=elapsed_rounded))   #返回 hh:mm:ss 形式的时间

In [54]:
def train1(model, optimizer):
    t0 = time.time()
    avg_loss, avg_acc = [],[]   
    model.train()
    for step, batch in enumerate(train_dataloader):
        # 每隔40个batch 输出一下所用时间.
        if step % 40 == 0 and not step == 0:            
            elapsed = format_time(time.time() - t0)             
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print(np.array(avg_loss).mean())

        b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device), batch[2].long().to(device)
        
        #output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        output = model(b_input_ids,  attention_masks=b_input_mask, labels=b_labels)
        loss, logits = output[0], output[1] 
    
        avg_loss.append(loss.item())
       
        acc = binary_acc(logits, b_labels)
        avg_acc.append(acc)
        
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 1.0)      #大于1的梯度将其设为1.0, 以防梯度爆炸
        optimizer.step()              #更新模型参数
        scheduler.step()              #更新learning rate
        
    avg_acc = np.array(avg_acc).mean()
    avg_loss = np.array(avg_loss).mean()
    return avg_loss, avg_acc

In [55]:
train1(model,optimizer)

  Batch    40  of    730.    Elapsed: 0:00:03.
1.7222979992628098
  Batch    80  of    730.    Elapsed: 0:00:07.
1.6926156148314475
  Batch   120  of    730.    Elapsed: 0:00:10.
1.6730328619480133
  Batch   160  of    730.    Elapsed: 0:00:13.
1.6607663802802564
  Batch   200  of    730.    Elapsed: 0:00:16.
1.646398152112961
  Batch   240  of    730.    Elapsed: 0:00:19.
1.635997615257899
  Batch   280  of    730.    Elapsed: 0:00:22.
1.6323858959334236
  Batch   320  of    730.    Elapsed: 0:00:25.
1.6270862191915512
  Batch   360  of    730.    Elapsed: 0:00:28.
1.6214422232574888
  Batch   400  of    730.    Elapsed: 0:00:31.
1.617201294898987
  Batch   440  of    730.    Elapsed: 0:00:34.
1.6116095960140229
  Batch   480  of    730.    Elapsed: 0:00:38.
1.606430250654618
  Batch   520  of    730.    Elapsed: 0:00:41.
1.6010217861487315
  Batch   560  of    730.    Elapsed: 0:00:44.
1.5965163714119366
  Batch   600  of    730.    Elapsed: 0:00:47.
1.5934633298714955
  Batch   640 

(1.586331417462597, 0.5307458143074582)

In [56]:
def evaluate(model):    
    avg_acc = []    
    model.eval()         #表示进入测试模式
      
    with torch.no_grad():
        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device), batch[2].long().to(device)
        
            output = model(b_input_ids, attention_masks=b_input_mask)
            
            acc = binary_acc(output[1], b_labels)
            avg_acc.append(acc)
    avg_acc = np.array(avg_acc).mean()
    return avg_acc

In [58]:
test_acc = evaluate(model)
print("epoch={},测试准确率={}".format(1, test_acc))

epoch=1,测试准确率=0.5495426829268293


In [37]:
def text_data_preprocessing(file):
    import os
    path = file
    files= os.listdir(path)
    txts = []
    for file in files: #遍历文件夹
        position = path+'\\'+ file #构造绝对路径，"\\"，其中一个'\'为转义符         
        with open(position, "r",encoding='utf-8') as f:    #打开文件
            data = f.read()   #读取文件
            txts.append(data)
    txts = ','.join(txts)#转化为非数组类型
    sentence = []
    j = 0
    for i in range(1,len(txts)):
        if txts[i]=='.' or txts[i]=='?':
            sentence.append(txts[j:i+1])
            j=i+1 
    return sentence

In [38]:
text_sentence = text_data_preprocessing('.\\test')

In [40]:
text_sentence=lower(text_sentence)

In [41]:
input_ids_test = []
attention_masks_test = []

# For every sentence...
for sent in text_sentence:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_test.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_test.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
#labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', text_sentence[1])
print('Token IDs:', input_ids_test[1])

Original:   or, when you were studying for a math test, did you ever ask your parents or sibling about different ways to tackle a certain problem?
Token IDs: tensor([  101,  2030,  1010,  2043,  2017,  2020,  5702,  2005,  1037,  8785,
         3231,  1010,  2106,  2017,  2412,  3198,  2115,  3008,  2030, 22941,
         2055,  2367,  3971,  2000, 11147,  1037,  3056,  3291,  1029,   102,
            0,     0])


C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [42]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test)
testdataloader = DataLoader(dataset_test, batch_size=16)

In [43]:
def test(model):     
    model.eval()         #表示进入测试模式
    output_list = []
    with torch.no_grad():
        for batch in testdataloader:
            b_input_ids, b_input_mask = batch[0].long().to(device), batch[1].long().to(device)
            output = model(b_input_ids, attention_masks=b_input_mask)
            output = torch.argmax(output[1], dim=1)
            output_list.append(output)
    return output_list

In [46]:
test(model)

[tensor([5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 5, 5, 2, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6], device='cuda:0'),
 tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 5, 0], device='cuda:0'),
 tensor([6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 6, 6, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 6], device='cuda:0'),
 tensor([2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5], device='cuda:0'),
 tensor([5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5], device='cuda:0')]